In [10]:
from pathlib import Path
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

desired_size = 118 # desired size is 128 but I want a 5 pixel border around the images in all directions so the images don't go to the end
directory_in_str = "C:/Users/nparker/Downloads/gear_images"

In [11]:
def normalize(arr):
    """
    Linear normalization
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    """
    arr = arr.astype('float')
    
    minval = arr.min()
    maxval = arr.max()
    if minval != maxval:
        arr -= minval
        arr *= (255.0/(maxval-minval))
    return arr

In [12]:
pathlist = Path(directory_in_str).glob('**/*.jp*')
for path in pathlist:
    # because path is object not string
    im_pth = str(path)
    im = Image.open(im_pth)

    old_size = im.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = im.resize(new_size, Image.ANTIALIAS)

    fill_color = "#ffffff" #white background

    # create a new image and paste the resized on it
    new_pil_im = Image.new("RGB", (desired_size, desired_size), fill_color)
    new_pil_im.paste(im, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))
    
    ImageOps.expand(new_pil_im,border=5,fill='white').save(im_pth) # add a 5 pixel border around the image

    simg = Image.open(im_pth)
    dimg = ImageOps.autocontrast(simg, cutoff=0.2) #replace the image with an auto contrasted version
    dimg.save(im_pth)
    
    img = cv2.imread(im_pth)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # open cv reads in as BGR channel this image was RGB 
    dst = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21) # remove the noise from the background of the image
    cv2.imwrite(im_pth,dst)
    
    im = Image.open(im_pth)
    new_img = Image.fromarray(normalize(np.array(im)).astype('uint8'),'RGB') #normalize the pixels in the image
    new_img.save(im_pth)
    
    simg = Image.open(im_pth)
    flipimg = simg.transpose(Image.FLIP_LEFT_RIGHT)
    flipimg.save(im_pth+"1.jpg")
    flipimg = simg.rotate(90)
    flipimg.save(im_pth+"2.jpg")
    
    